#### 1. 什么是GLOVE

&emsp;&emsp;Global Vectors for Word Representation，是一个基于**全局词频统计**的词表征工具。可以把一个单词表达成一个由实数组成的向量，这些向量捕捉到了单词之间的一些语义特征，通过对向量的运算，可以计算出两个单词之间的语义相似性。

#### 2.Glove如何实现

1. 根据预料库构建一个共现矩阵X,矩阵中的每一个元素$X_{ij}$代表center word单词i和上下文单词j在特定大小的context window内共同出现的次数。glove根据两个单词在上下文窗口的距离d，提出了一个衰减函数:decay = 1/d用于计算权重，也就是说距离越远的两个单词所占总计数的权重越小。

2. 构建词向量(Word Vector)和共现矩阵的之间的近似关系：

$$w_{i}^{T} \overline{W_{j}} + b_{i} + \overline{b_{j}} = log(X_{ij}) \tag{1}$$

&emsp;&emsp;其中，$w_{i}^{T}$和$\overline{W_{j}}$是我们最终求解的词向量，$b_{i}$ 和$\overline{b_{j}}$ 分别是两个词向量的bias term，可能会有几个疑问：怎么来的，为什么使用两个词向量。

3. 构建loss function

$$J = \sum_{i=1,j=1}^{V}{f(X_{ij})(w_{i}^{T} \overline{W_{j}} + b_{i} + \overline{b_{j}} - log(X_{ij}))^{2}} \tag{2}$$

&emsp;&emsp;这个loss function 的基本形式就是最简单的mean square loss，只不过在此基础上加了一个权重函数$f(X_{ij})$，在一个预料库中，肯定存在很多单词是在一起出现的次数是很多的。那么我们希望：


    3.1.这些单词的权重要大于那些很少在一起出现的单词，所以这个函数应该是非递减的函数
    3.2.不希望这个权重过大，当到达一定程度之后应该不再增加
    3.3.如果两个单词没有在一起出现过，也就是$X_{ij}=0$，那么他们不应该参与到loss function中，即f(0)=0

&emsp;&emsp;作者采用了如下形式的分段函数:

$$ f(x)=\begin{cases}
(x/x_{max})^{\alpha} & if x < x_{max} \\
1 & otherwise
\end{cases} \tag{3}$$

&emsp;&emsp;在论文中的所有实验中，$\alpha$的取值都是0.75，而$x_{max}$的取值都是100。

#### 3.公式推导

- $X_{ij}$表示单词j出现在单词i的上下文中的次数
- $X_{i}$表示单词i一共是多少个其他词的中心词 $X_{i} = \sum_{j=1}^{N}{X_{ij}}$
- $P_{ij} = P(j|i) = X_{ij}/X_{j}$表示单词j出现在单词i的上下文中的概率
- $Ratio_{i,j,k} = \frac{P_{i,k}}{P_{j,k}}$表示词k出现在词i和出现在词j附近比例的比例

&emsp;&emsp;然后作者发现一个规律：

$$ ratio_{i,j,k} =\begin{cases}
趋近1 & 单词i,k相关 单词j,k相关 \\
很大 & 单词i,k相关 单词j,k不相关 \\
很小 & 单词i,k不相关 单词j,k相关 \\
趋近1 & 单词i,k不相关 单词j,k不相关 
\end{cases} \tag{4}$$

&emsp;&emsp;然后作者就设计目标函数去拟合这个规律，这个ratio跟i,j,k有关，所以将词i,j,k的向量放进函数中，代价函数的模板应该是:

$$J = \sum_{i,j,k}^{N}{(\frac{P_{i,k}}{P_{j,k}} - g(v_{i},v_{j},\overline{v_{k}}))} \tag{5}$$

&emsp;&emsp;为了捕捉上面的概率比例，构造如下函数

$$ g(v_{i},v_{j},\overline{v_{k}}) = \frac{P_{i,k}}{P_{j,k}} \tag{6}$$

&emsp;&emsp;其中，函数g的参数和具体形式未定，有三个参数$v_i,v_j,\overline{v_{k}}$,其中$v,\overline{v}$是不同的向量。

&emsp;&emsp;因为向量空间是线性结构的，所以要表达出两个概率的比例差，最简单的办法是做差，于是得到

$$g(v_i - v_j,\overline{v_k}) = \frac{P_{i,k}}{P_{j,k}} \tag{7}$$

&emsp;&emsp;公式7右侧是一个数量，而左侧是一个向量，于是将左侧转换成两个向量的内积形式:

$$g((v_i - v_j)^{T} \overline{v_k}) = \frac{P_{i,k}}{P_{j,k}} \tag{8}$$

&emsp;&emsp;注意到共现矩阵是一个对称的矩阵，即$X == X^{T}$,把$v_i$成为中心单词，$\overline{w_k}$称为$w_i$的上下文的某个单词，因此有$P_{i,j} == P_{j,i}$，从某种角度看,$\overline{w}$和w的角色是可以互换的，希望g函数能隐含这种特性,再看看公式8:

$$g((v_i - v_j)^{T} \overline{v_k}) = \frac{P_{i,k}}{P_{j,k}} != \frac{P_{k,i}}{P_{k,j}} \tag{9}$$

&emsp;&emsp;于是论文中提出这样一种关系。

$$g((v_i - v_j)^{T} \overline{v_k}) = \frac{g((v_i)^{T} \overline{v_k})}{g((v_j)^{T} \overline{v_k})} == \frac{g((\overline{v_k})^{T} v_i)}{g((\overline{v_k})^{T} v_j)} \tag{10}$$

&emsp;&emsp;结合公式7，可以得到:
$$F(w_{i}^{T} \overline{w_k}) = P_{i,k} = \frac{X_{i,k}}{X_i} \tag{11}$$

&emsp;&emsp;另F = exp，可以得到:

$$w_{i}^{T} \overline{w_k} = log(P_{i,k}) = log(X_{i,k}) - log(X_i) \tag{12}$$

&emsp;&emsp;此时我们发现等号右侧的$log(X_i)$的存在，公式12是不满足对称性的。这个$log(X_i)$其实跟k独立的，只跟i相关，于是可以针对$v_i$增加一个bias term $b_i$将他替换掉，于是有:

$$v_i^T \overline{w_k} + b_i = log(X_{i,k}) \tag{13}$$

&emsp;&emsp;此时公式13还是不满足对称性，于是我们针对$v_k$增加一个bias term $b_k$，从而得到公式1的形式:

$$v_i^T \overline{v_k} + b_i + b_k = log(X_{i,k}) \tag{14}$$


#### 4.参考文献
1. [论文分享-->GloVe: Global Vectors for Word Representation](https://blog.csdn.net/mr_tyting/article/details/80180780)
2. [词向量经典模型：从word2vec、glove、ELMo到BERT](https://zhuanlan.zhihu.com/p/51682879)
3. [GloVe详解](http://www.fanyeong.com/2018/02/19/glove-in-detail/)

&emsp;&emsp;

In [2]:
input_str = "i love you but you love him"
word_index = {}
for w in input_str.split():
    if w not in word_index:
        word_index[w] = len(word_index)
        
input_list = input_str.split()
input_len = len(input_list)

X = [[0 for x in range(len(word_index))] for i in range(len(word_index))]

window_size = 2
for c_i,center_word in enumerate(input_list):
    for pos_i in range(-window_size,window_size+1):
        context_i = c_i + pos_i
        if context_i <0 or context_i >= input_len:continue
        context_word = input_list[context_i]
        
        # 获取word 的id
        center_index = word_index[center_word]
        context_index = word_index[context_word]
        X[center_index][context_index] += 1        

In [3]:
X

[[1, 1, 1, 0, 0],
 [1, 2, 2, 2, 1],
 [1, 2, 4, 2, 1],
 [0, 2, 2, 1, 0],
 [0, 1, 1, 0, 1]]

In [7]:
import numpy as np

X_array = np.array(X)

In [15]:
X_array

array([[1, 1, 1, 0, 0],
       [1, 2, 2, 2, 1],
       [1, 2, 4, 2, 1],
       [0, 2, 2, 1, 0],
       [0, 1, 1, 0, 1]])

In [18]:
X_array.sum(axis=1,keepdims=True)

array([[ 3],
       [ 8],
       [10],
       [ 5],
       [ 3]])

In [19]:
P_array = X_array/X_array.sum(axis=1,keepdims=True)

In [20]:
P_array

array([[0.33333333, 0.33333333, 0.33333333, 0.        , 0.        ],
       [0.125     , 0.25      , 0.25      , 0.25      , 0.125     ],
       [0.1       , 0.2       , 0.4       , 0.2       , 0.1       ],
       [0.        , 0.4       , 0.4       , 0.2       , 0.        ],
       [0.        , 0.33333333, 0.33333333, 0.        , 0.33333333]])